In [ ]:
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Vertex AI模型花园 - LaMa

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/model_garden/model_garden_pytorch_lama.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在Colab中运行
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/model_garden/model_garden_pytorch_lama.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在GitHub上查看
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/notebooks/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/community/model_garden/model_garden_pytorch_lama.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI标志">
在Vertex AI工作台中打开
    </a>
    (建议使用Python-3 CPU笔记本)
  </td>
</table>

## 概述

这个笔记本演示了在Vertex AI中部署预构建的[LaMa](https://github.com/advimman/lama)模型。

### 目标

- 将预构建的LaMa模型部署到Vertex Endpoint，并对其进行查询。

### 成本

本教程使用了Google Cloud的计费组件：

* Vertex AI

了解 [Vertex AI定价](https://cloud.google.com/vertex-ai/pricing) 并使用 [定价计算器](https://cloud.google.com/products/calculator/) 根据您的预计使用情况生成成本估算。

## 在开始之前

**注意**：Jupyter会将以`!`前缀开头的行视为shell命令，并且会将以`$`前缀开头的Python变量插入到这些命令中。

### 设立笔记本

只适用于Colab
如果您正在使用Workbench，请跳过此部分，并运行以下命令。

In [ ]:
if "google.colab" in str(get_ipython()):
    ! pip3 install --upgrade google-cloud-aiplatform
    from google.colab import auth as google_auth

    google_auth.authenticate_user()
    ! pip3 install --upgrade pip

    # Restart the notebook kernel after installs.
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

### 设置 Google Cloud 项目

1. [选择或创建一个 Google Cloud 项目](https://console.cloud.google.com/cloud-resource-manager)。您首次创建帐户时，将获得一笔为计算/存储成本提供的300美元免费信用额度。

1. [确保您的项目已启用计费](https://cloud.google.com/billing/docs/how-to/modify-project)。

1. [启用 Vertex AI API 和 Compute Engine API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com,compute_component)。

1. [创建一个云存储存储桶](https://cloud.google.com/storage/docs/creating-buckets) 用于存储实验输出。

1. [创建一个服务账号](https://cloud.google.com/iam/docs/service-accounts-create#iam-service-accounts-create-console)，并为其分配`Vertex AI User` 和 `Storage Object Admin` 角色，以便将经过优化的模型部署到 Vertex AI 终端点。

填写变量以进行实验环境：

In [ ]:
# Cloud project id.
PROJECT_ID = ""  # @param {type:"string"}

# The region you want to launch jobs in.
REGION = "us-central1"  # @param {type:"string"}

# The Cloud Storage bucket for storing experiments output.
GCS_BUCKET = "gs://"  # @param {type:"string"}

# The service account for deploying fine tuned model.
# The service account looks like:
# '<account_name>@<project>.iam.gserviceaccount.com'
SERVICE_ACCOUNT = ""  # @param {type:"string"}

初始化 Vertex-AI API

In [ ]:
from google.cloud import aiplatform

aiplatform.init(project=PROJECT_ID, location=REGION, staging_bucket=GCS_BUCKET)

定义常量

In [ ]:
# The pre-built serving docker image. It contains serving scripts and models.
SERVE_DOCKER_URI = "us-docker.pkg.dev/vertex-ai/vertex-vision-model-garden-dockers/lama-serve:20240125_0903_RC00"

### 定义常用功能

In [ ]:
import base64
from datetime import datetime
from io import BytesIO
from typing import List, Tuple

import requests
from google.cloud import aiplatform
from PIL import Image


def create_job_name(prefix: str) -> str:
    """Return a timestamped string."""
    now = datetime.now().strftime("%Y%m%d_%H%M%S")
    job_name = f"{prefix}-{now}"
    return job_name


def download_image(url: str) -> Image.Image:
    """Get image given a URL."""
    response = requests.get(url)
    return Image.open(BytesIO(response.content))


def image_to_base64(image: Image.Image, format="JPEG") -> str:
    """Convert an image to its base64 representation."""
    buffer = BytesIO()
    image.save(buffer, format=format)
    image_str = base64.b64encode(buffer.getvalue()).decode("utf-8")
    return image_str


def base64_to_image(image_str: str) -> Image.Image:
    """Convert an image from its base64 representation."""
    image = Image.open(BytesIO(base64.b64decode(image_str)))
    return image


def image_grid(imgs: List[Image.Image], rows: int = 2, cols: int = 2):
    """Display images in a grid."""
    w, h = imgs[0].size
    grid = Image.new("RGB", size=(cols * w, rows * h))
    for i, img in enumerate(imgs):
        grid.paste(img, box=(i % cols * w, i // cols * h))
    return grid


def deploy_model(
    model_name: str,
    machine_type: str = "g2-standard-24",
    accelerator_type: str = "NVIDIA_L4",
    accelerator_count: int = 2,
) -> Tuple[aiplatform.Model, aiplatform.Endpoint]:
    """Upload a model to Model registry and deploy it to a Vertex Endpoint."""
    endpoint = aiplatform.Endpoint.create(display_name=f"{model_name}-endpoint")
    serving_env = {"MODEL_ID": "lama", "DEPLOY_SOURCE": "notebook"}

    model = aiplatform.Model.upload(
        display_name=model_name,
        serving_container_image_uri=SERVE_DOCKER_URI,
        serving_container_ports=[7080],
        serving_container_predict_route="/predictions/lama",
        serving_container_health_route="/ping",
        serving_container_environment_variables=serving_env,
    )
    model.deploy(
        endpoint=endpoint,
        machine_type=machine_type,
        accelerator_type=accelerator_type,
        accelerator_count=accelerator_count,
        deploy_request_timeout=1800,
        service_account=SERVICE_ACCOUNT,
    )
    return model, endpoint

## 下载示例图像

我们下载了官方[LaMa GitHub项目](https://github.com/advimman/lama/tree/main/)提供的示例图像和蒙版，用于预测。

In [ ]:
# Download and unzip images and masks.
!gdown --fuzzy https://drive.google.com/file/d/1p3g1XWECRuybw423aKWmToi6YrjZWq3n/view?usp=drive_link
!unzip LaMa_test_images.zip

# List available test images and masks
!ls LaMa_test_images

上传并部署模型

这一部分将LaMa模型上传到模型注册表，并将其部署在终端上。 

当部署在两个L4 GPU上时，一个请求的平均推断时间约为15秒。 

模型部署步骤将需要约15分钟才能完成。

In [ ]:
model, endpoint = deploy_model(
    model_name=create_job_name("lama"),
)

注意：模型权重将在部署成功后下载。因此，在上述模型部署步骤成功之后，需要额外等待5分钟，然后才能运行下面的下一步。否则，当您向端点发送请求时，您可能会看到“ServiceUnavailable: 503 502:Bad Gateway”错误。

加载并显示测试图片和蒙版。

In [ ]:
init_image_name = "bench2"  # @param {type:"string"}
init_mask_name = "bench2_mask"  # @param {type:"string"}

init_image = Image.open(f"LaMa_test_images/{init_image_name}.png")
mask_image = Image.open(f"LaMa_test_images/{init_mask_name}.png")

display(init_image)
display(mask_image)

将测试图像和蒙版发送到端点。

In [ ]:
instances = [
    {
        "image": image_to_base64(init_image),
        "mask": image_to_base64(mask_image),
        "refine": True,
    },
]
response = endpoint.predict(instances=instances)
output_image = [base64_to_image(image) for image in response.predictions][0]
display(output_image)

清理资源

In [ ]:
# Undeploy model and delete endpoint.
endpoint.delete(force=True)

# Delete models.
model.delete()